In [14]:
import boto3
import json
import os

state_path = "boardgamegeek.tfstate"
temp_import_file = "temp_import.tf"
state_type = "aws_ecr_repository"
output_file = "ecr_imported.tf"

create import blocks for Terraform

In [5]:
ecr_client = boto3.client('ecr')
repositories = ecr_client.describe_repositories()['repositories']
repo_names = [f['repositoryName'] for f in repositories]

In [6]:
if os.path.exists(temp_import_file):
    os.remove(temp_import_file)

for repo_name in repo_names:
    import_block = f'resource "{state_type}" "{repo_name}" {{\nname = "{repo_name}"\n}}\n'
    # Write the HCL configuration to a .tf file
    with open(temp_import_file, 'a') as tf_file:
        tf_file.write(import_block)

In [ ]:
os.system("terraform init")

In [ ]:
for repo_name in repo_names:
    import_command = f"terraform import {state_type}.{repo_name} {repo_name}"
    os.system(import_command)

Open imported blocks and extract settings

In [ ]:
data = open(state_path).read()
state = json.loads(data)['resources']
specific_types = [f for f in state if f['type'] == state_type]
specific_types

In [ ]:
for resource in specific_types:
    instance = resource['instances'][0]['attributes']

    # Define the HCL configuration for the resource
    hcl = f"""
resource "aws_ecr_repository" "{resource['name']}" {{
    name                      = "{instance['name']}"
    image_tag_mutability       = "{instance['image_tag_mutability']}"
    image_scanning_configuration {{
        scan_on_push = {str(instance['image_scanning_configuration'][0]['scan_on_push']).lower()}
    }}
    encryption_configuration {{
        encryption_type = "{instance['encryption_configuration'][0]['encryption_type']}"
        kms_key         = "{instance['encryption_configuration'][0]['kms_key']}"
    }}

    tags = {{
        {', '.join([f'"{key}" = "{value}"' for key, value in instance['tags'].items()])}
    }}
}}
    """

    # Write the HCL configuration to a .tf file
    with open(output_file, 'a') as tf_file:
        tf_file.write(hcl)

    print(f"HCL configuration written to {output_file} for {resource['name']}")

if os.path.exists(temp_import_file):
    os.remove(temp_import_file)